# Setup

In [2]:
import pandas as pd
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

C:\Users\ashok\DemoVirtualEnv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Data

In [3]:
# usign pakistan intellectual capital dataset
professor_info = pd.read_csv('pakistan_intellectual_capital.csv')

In [4]:
# checking the data
professor_info.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [5]:
# lets choose a column to clean up for example 'Country'
countries = professor_info['Country'].unique()

# sorting alphabetically
countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

In [6]:
# we can see alot of inconsistent data above
# converting to lower case
professor_info['Country'] = professor_info['Country'].str.lower()
# we need to remove white spaces
professor_info['Country'] = professor_info['Country'].str.strip()

# Using Fuzzy matching

In [7]:
countries = professor_info['Country'].unique()

# sorting alphabetically
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

In [8]:
# we can still see the inconsistent with 'south korea' and 'southkore'
# using fuzzy to get top 10 closest matches to 'south korea'
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer= fuzzywuzzy.fuzz.token_sort_ratio)

#printing
matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('ireland', 33),
 ('portugal', 32),
 ('singapore', 30),
 ('netherland', 29),
 ('macau', 25),
 ('usofa', 25)]

In [18]:
# creating the function to replace the inconsistency
def replace_matches(df, column, string_to_match, min_ratio = 47):
    # getting unique columns
    strings = df[column].unique()
    # getting top 10 matches with fuzzy to string_to_match
    matches = fuzzywuzzy.process.extract(string_to_match, strings, limit=10, scorer= fuzzywuzzy.fuzz.token_sort_ratio)
    #finding the close_matches according to min_ratio
    close_matches = [matches[0] for matches in matches if matches[1]>=min_ratio]
    #replacing the rows with close matches
    rows_with_matches = df[column].isin(close_matches)
    df.loc[rows_with_matches, column] = string_to_match
    
    # message
    print('completed')

In [19]:
replace_matches(df=professor_info, column='Country', string_to_match='south korea')

completed


In [20]:
# lets check the function worked or not
countries = professor_info['Country'].unique()

# sorting alphabetically
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)